In [1]:
#lets retrieve time steps for a model

from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import torch
import datasets

# import mddel, featureextractot, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
#load first sample of English common-voice
dataset = load_dataset("common_voice","en",split="train", streaming=True)
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=16_000))
dataset_iter = iter(dataset)
sample = next(dataset_iter)

KeyboardInterrupt: 

In [ ]:
# forward sample through model to get greedily predicted transcripts ids
input_values = feature_extractor(sample["audio"]["array"], return_tensors="pt").input_values
logits = model(input_values).logits[0]
pred_ids = torch.argmax(logits, axis=-1)

In [ ]:
# retrieve word stamps (analog commands for output_char_offsets)
outputs = tokenizer.decode(pred_ids, output_word_offset=True)

In [ ]:
# compute time_offset in seconds as a product of downsampling ration and sampling rate
time_offsest = model.config.inputs_to_logitd_ratio/ feature_extractor.sampling_rate

eord_offsets = [
    {
        "word":d["word"],
        "start_time":round(d["start_offset"]*time_offsest,2)
        "end_time": round(d["end_offset"]*time_offsest,2)
    }
    for d in outputs.word_offsets
]

In [ ]:
# compare word offsets with audio common_voice_en_100038.mp3 online on the dataset viewer:


Code

In [1]:
from transformers import pipeline
from pprint import pprint

#create model using pipeline for question answering
model_unmasker = pipeline('fill-mask')

#define the content for the question
pprint(model_unmasker(f"I have never watched something like this {unmasker.tokenizer.mask_token} and it was amazing."))


No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


ReadTimeout: HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10.0)